In [1]:
from glob import glob
import json
import os
import pandas as pd
import re

os.chdir("../")

In [43]:
path = "data/Italy/2022\\010101101110812620.txt"

year = 2022

In [3]:
output = []
output_location = []
for path in glob(f"data/Italy/{year}/*.txt"):
    
    try:

        file = open(path, "r", encoding="utf-8")
        data = json.load(file)
        details = data["int"]
        cand = data["cand"]

        file.close()

        polling_id = re.findall(r"[^\/]+(?=\.)", path)[0][5:]

        tmp = pd.DataFrame(cand).explode("liste")

        df = pd.json_normalize(tmp["liste"])
        df["polling_id"] = polling_id
        df = df.rename(columns={
            "desc_lis": "candidate",
            "voti": "value"
        })

        output.append(df)
        output_location.append({
            "commune_id": details["cod_com"],
            "commune": details["desc_com"],
            "province_id": details["cod_prov"],
            "province": details["desc_prov"],
            "polling_id": polling_id,
            "circunscription_id": details["cod_circ"],
            "circunscription": details["desc_circ"],
            "dt_agg": details["dt_agg"]
        })
        
    except:
#         pass
        print(path)
    
df = pd.concat(output, ignore_index=True)
df.head()

data/Italy/2022\022700027010040010.txt
data/Italy/2022\022700027010040020.txt
data/Italy/2022\022700027010040030.txt
data/Italy/2022\022700027010040040.txt
data/Italy/2022\022700027010040050.txt
data/Italy/2022\022700027010040060.txt
data/Italy/2022\022700027010040070.txt
data/Italy/2022\022700027010040080.txt
data/Italy/2022\022700027010040090.txt
data/Italy/2022\022700027010040100.txt
data/Italy/2022\022700027010040110.txt
data/Italy/2022\022700027010040120.txt
data/Italy/2022\022700027010040130.txt
data/Italy/2022\022700027010040140.txt
data/Italy/2022\022700027010040150.txt
data/Italy/2022\022700027010040160.txt
data/Italy/2022\022700027010040170.txt
data/Italy/2022\022700027010040180.txt
data/Italy/2022\022700027010040190.txt
data/Italy/2022\022700027010040200.txt
data/Italy/2022\022700027010040210.txt
data/Italy/2022\022700027010040220.txt
data/Italy/2022\022700027010040230.txt
data/Italy/2022\022700027010040240.txt
data/Italy/2022\022700027010040250.txt
data/Italy/2022\022700027

,pos,candidate,img_lis,value,perc,polling_id
0,13,PARTITO DEMOCRATICO - ITALIA DEMOCRATICA E PRO...,2-20220925000000-13.png,51930,"26,22",010101101110812620
1,17,ALLEANZA VERDI E SINISTRA,2-20220925000000-17.png,14139,"7,14",010101101110812620
2,16,+EUROPA,2-20220925000000-16.png,12048,"6,08",010101101110812620
3,9,IMPEGNO CIVICO LUIGI DI MAIO - CENTRO DEMOCRATICO,2-20220925000000-9.png,813,"0,41",010101101110812620
4,14,FRATELLI D'ITALIA CON GIORGIA MELONI,2-20220925000000-14.png,39003,"19,69",010101101110812620


In [4]:
df_filtered = df[["value", "polling_id", "candidate"]].copy()
df_filtered["rank"] = df_filtered.groupby(["polling_id"])["value"].rank("min", ascending=False).astype(int)

tt = df_filtered.groupby(["polling_id", "candidate"]).agg({"value": "sum"})
tt["rate"] = tt.groupby(level=[0]).apply(lambda x: x/x.sum())
tt = tt.reset_index()
tt = tt[["polling_id", "candidate", "rate"]]

df_filtered = pd.merge(df_filtered, tt, on=["polling_id", "candidate"])
df_filtered.head()

C:\Users\cnava\AppData\Local\Temp\ipykernel_14948\3505218643.py:5: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  tt["rate"] = tt.groupby(level=[0]).apply(lambda x: x/x.sum())


,value,polling_id,candidate,rank,rate
0,51930,010101101110812620,PARTITO DEMOCRATICO - ITALIA DEMOCRATICA E PRO...,1,0.262187
1,14139,010101101110812620,ALLEANZA VERDI E SINISTRA,5,0.071386
2,12048,010101101110812620,+EUROPA,6,0.060829
3,813,010101101110812620,IMPEGNO CIVICO LUIGI DI MAIO - CENTRO DEMOCRATICO,14,0.004105
4,39003,010101101110812620,FRATELLI D'ITALIA CON GIORGIA MELONI,2,0.196920


In [6]:
df_filtered.to_csv(f"data_output/Italy/{year}_first_round.csv.gz", compression="gzip", index=False)

In [35]:
df_nuts = pd.read_excel(
    "https://ec.europa.eu/eurostat/documents/345175/629341/NUTS2021.xlsx",
    sheet_name="NUTS & SR 2021"
)
df_nuts["country_code"] = df_nuts["Code 2021"].str[0:2]

df_nuts = df_nuts[df_nuts["country_code"] == "IT"]#.fillna(method="ffill")

for col in ["NUTS level 1", "NUTS level 2"]:
    df_nuts[col] = df_nuts[col].fillna(method="ffill")
    
df_nuts.head()

def get_nuts_level(df, level=1):
    output_df = df[["Code 2021", f"NUTS level {level}", "NUTS level"]].dropna()
    output_df = output_df[output_df["NUTS level"] == level]
    output_df = output_df[["Code 2021", f"NUTS level {level}"]]
    output_df = output_df.rename(columns={"Code 2021": f"nuts_{level}", f"NUTS level {level}": f"nuts_{level}_name"})
    output_df = output_df.reset_index(drop=True)
    output_df[f"nuts_{level}_name"] = output_df[f"nuts_{level}_name"].str.upper()

    return output_df

df_nuts_1 = get_nuts_level(df_nuts, level=1)
df_nuts_2 = get_nuts_level(df_nuts, level=2)

df_nuts_3 = df_nuts[["Code 2021", "NUTS level 1", "NUTS level 2", "NUTS level 3"]].dropna()
df_nuts_3 = df_nuts_3.rename(
    columns={
        "Code 2021": "nuts_3", 
        "NUTS level 3": "department",
        "NUTS level 1": "nuts_1_name",
        "NUTS level 2": "nuts_2_name",
        "NUTS level 3": "nuts_3_name"
    })

for lv in range(1, 3 + 1):
    df_nuts_3[f"nuts_{lv}_name"] = df_nuts_3[f"nuts_{lv}_name"].str.upper()

df_nuts_3 = pd.merge(df_nuts_3, df_nuts_2, on="nuts_2_name")
df_nuts_3 = pd.merge(df_nuts_3, df_nuts_1, on="nuts_1_name")
df_nuts_3["nuts_3_name"] = df_nuts_3["nuts_3_name"].str.strip().str.replace("’", "'")

df_nuts_3.head()

,nuts_3,nuts_1_name,nuts_2_name,nuts_3_name,nuts_2,nuts_1
0,ITC11,NORD-OVEST,PIEMONTE,TORINO,ITC1,ITC
1,ITC12,NORD-OVEST,PIEMONTE,VERCELLI,ITC1,ITC
2,ITC13,NORD-OVEST,PIEMONTE,BIELLA,ITC1,ITC
3,ITC14,NORD-OVEST,PIEMONTE,VERBANO-CUSIO-OSSOLA,ITC1,ITC
4,ITC15,NORD-OVEST,PIEMONTE,NOVARA,ITC1,ITC


In [38]:
df_nuts_3["nuts_3_name"] = df_nuts_3["nuts_3_name"].replace({
    "BOLZANO-BOZEN": "BOLZANO",
    "FORLÌ-CESENA": "FORLI'-CESENA",
    "REGGIO DI CALABRIA": "REGGIO CALABRIA",
    "REGGIO NELL'EMILIA": "REGGIO NELL' EMILIA"
})

In [41]:
set(df_nuts_3.nuts_3_name) - set(df_location.province)

{'EXTRA-REGIO NUTS 3', "VALLE D'AOSTA/VALLÉE D'AOSTE"}

In [39]:
set(df_location.province) - set(df_nuts_3.nuts_3_name)

set()

In [44]:
df_location = pd.DataFrame(output_location)
df_location = pd.merge(df_location, df_nuts_3, left_on="province", right_on="nuts_3_name", how="left")
df_location.to_csv(f"data_output/Italy/{year}_first_round_location.csv.gz", compression="gzip", index=False)

In [9]:
df_location

,commune_id,commune,province_id,province,polling_id,circunscription_id,circunscription,dt_agg
0,2620,TORINO,81,TORINO,010101101110812620,1,PIEMONTE 1,20220926073040
1,2620,TORINO,81,TORINO,010101101120812620,1,PIEMONTE 1,20220926071121
2,80,ALPIGNANO,81,TORINO,010101101130810080,1,PIEMONTE 1,20220926044826
3,280,BORGARO TORINESE,81,TORINO,010101101130810280,1,PIEMONTE 1,20220926093357
4,450,BUTTIGLIERA ALTA,81,TORINO,010101101130810450,1,PIEMONTE 1,20220926043856
...,...,...,...,...,...,...,...,...
7837,730,URI,73,SASSARI,202626126140730730,26,SARDEGNA,20220926050243
7838,740,USINI,73,SASSARI,202626126140730740,26,SARDEGNA,20220926055054
7839,741,VALLEDORIA,73,SASSARI,202626126140730741,26,SARDEGNA,20220926042551
7840,742,VIDDALBA,73,SASSARI,202626126140730742,26,SARDEGNA,20220926054426
